In [ ]:
##########################################################################
######################                              ######################
######################          model 파트          ######################
######################                              ######################
##########################################################################

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostRegressor
from bayes_opt import BayesianOptimization
from IPython.display import clear_output


def catboost_model(feat_data, cat_features):
    """
    BayesianOptimization을 이용한 최적 파라미터 도출값
    """
    max_params = {
        'depth': int(13.96),
        'l2_leaf_reg': 2.853,
        'learning_rate': 0.1234,
        'max_bin': int(171.3),
        'model_size_reg': 1.161,
        'n_estimators': int(371.8),
        'subsample': 0.7814,
        'verbose': False,
        'bootstrap_type': 'Poisson',
        'task_type': 'GPU',
        'cat_features': cat_features
    }

    """
    train and forecast today's solds
    """
    tscv = TimeSeriesSplit(n_splits=10, test_size=30)
    scores = []
    date_idx = feat_data['date'].unique()
    for i, (train_date_idx, test_date_idx) in enumerate(tscv.split(date_idx)):
        if i < 9:
            continue
        train_date = date_idx[train_date_idx]
        test_date = date_idx[test_date_idx]
        print(f'> Split {i+1} ==================================')
        print(f'train({len(train_date)}) : {train_date[0]} ~ {train_date[-1]}')
        print(f'test({len(test_date)}) : {test_date[0]} ~ {test_date[-1]}')

        train = feat_data.set_index('date').loc[train_date, :]
    #     train_X = train.drop(columns=['item_id', 'year', 'target'])
        train_X = train.drop(columns=['item_id','target'])
        train_y = train.loc[:, 'target']

        model = CatBoostRegressor(**max_params)
        model.fit(train_X, train_y)

        test = feat_data.set_index('date').loc[test_date, :]
    #     test_X = test.drop(columns=['item_id', 'year', 'target'])
        test_X = test.drop(columns=['item_id','target'])
        test_y = test.loc[:, 'target']

        test['pred'] = model.predict(test_X)
        test = test.dropna()
        
        return test
